#PDF_Redaction System

Installation of Required Library
PyMuPDF(fitz)


In [1]:
!pip install PyMuPDF transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 25.6 MB/s eta 0:00:00


Importing Required Libraries

In [2]:
from google.colab import files
import fitz  # PyMuPDF
from transformers import pipeline
import re
import os

Initializing NER

In [3]:
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

defining Regex Pattern

In [4]:
patterns = {
    'phone': r'\b(\+?(\d{1,3})?[-.\s]??\(?\d{1,4}?\)?[-.\s]??\d{1,4}[-.\s]??\d{1,9})\b',
    'credit_card': r'\b(?:\d[ -]*?){13,16}\b',
    'ssn': r'\b\d{3}-\d{2}-\d{4}\b',
    'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
}

In [5]:
def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    text_pages = []
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        text = page.get_text("text")
        text_pages.append((page_num, text))
    return text_pages, pdf_document

In [6]:
def identify_entities(text):
    entities = ner_pipeline(text)
    pii_entities = [entity for entity in entities if entity['entity'] in ["B-PER", "I-PER", "B-ORG", "I-ORG"]]
    for label, pattern in patterns.items():
        matches = re.finditer(pattern, text)
        for match in matches:
            pii_entities.append({'word': match.group(), 'start': match.start(), 'end': match.end(), 'entity': label})

    return pii_entities

In [7]:
def redact_entities_in_pdf(pdf_document, pii_entities, page_texts):
    for page_num, page_text in page_texts:
        page = pdf_document[page_num]

        for entity in pii_entities:
            term = entity["word"]
            locations = page.search_for(term)
            for location in locations:
                page.add_redact_annot(location, fill=(0, 0, 0))
            page.apply_redactions()

In [8]:
def redact_pdf(pdf_path, output_path):
    text_pages, pdf_document = extract_text_from_pdf(pdf_path)
    all_entities = []
    for _, page_text in text_pages:
        pii_entities = identify_entities(page_text)
        all_entities.extend(pii_entities)

    redact_entities_in_pdf(pdf_document, all_entities, text_pages)
    pdf_document.save(output_path)
    pdf_document.close()
    print(f"Redacted PDF saved to {output_path}")

#Upload PDF file from user

In [9]:
print("Upload your PDF")
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
output_directory = "/content/redacted_documents"
os.makedirs(output_directory, exist_ok=True)
output_path = os.path.join(output_directory, "PII_redacted_output.pdf")

redact_pdf(pdf_path, output_path)

print("Redaction complete. Downloading the redacted file...")
files.download(output_path)

Upload your PDF


Saving RAGHAV KUMAR JHA Resume.pdf to RAGHAV KUMAR JHA Resume.pdf
Redacted PDF saved to /content/redacted_documents/PII_redacted_output.pdf
Redaction complete. Downloading the redacted file...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

****Further AI integration is left to Automate the PDF_Redaction System***
